In [14]:
import pandas as pd
import requests
import gmaps

from config import g_key

In [15]:
weather_data = pd.read_csv("cities.csv")
weather_data.head()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,Ostrovnoy,RU,1631725660,68.0531,39.5131,40.39,83,40,14.36
1,1,Mahébourg,MU,1631725660,-20.4081,57.7000,69.26,78,15,12.26
2,2,Qaanaaq,GL,1631725660,77.4840,-69.3632,16.70,74,11,9.31
3,3,Zhuhai,CN,1631725661,22.2769,113.5678,88.05,74,20,6.91
4,4,Cape Town,ZA,1631725661,-33.9258,18.4232,58.10,76,5,3.00


In [16]:
# Store latitude and longitude in locations
locations = weather_data[["Latitude", "Longitude"]]

hum_weight = weather_data["Humidity"]

In [17]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=hum_weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
narrowed_city_df = weather_data.loc[((weather_data['Max Temperature'] <= 80) & (weather_data['Max Temperature'] >= 70)
                                                 & (weather_data['Wind Speed'] < 10) &
                                                 (weather_data['Cloudiness'] == 0))]
narrowed_city_df

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
24,24,Fallon,US,1631725667,39.4735,-118.7774,79.45,20,0,4.47
46,46,Beloha,MG,1631725674,-25.1667,45.0500,72.21,57,0,6.76
93,93,Antakya,TR,1631725688,36.2066,36.1572,78.62,76,0,8.59
259,259,Kumul,CN,1631725737,42.8000,93.4500,76.33,17,0,8.52
296,296,Khāsh,IR,1631725748,28.2211,61.2158,78.80,8,0,7.63
350,350,Yangi Marg`ilon,UZ,1631725767,40.4272,71.7189,75.92,31,0,6.91


In [18]:
# Filter out the rows
hotel_df = narrowed_city_df.dropna(how = 'all')

In [19]:
hotel_df['Hotel Name'] = ""
hotel_df.reset_index(drop=True)

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,24,Fallon,US,1631725667,39.4735,-118.7774,79.45,20,0,4.47,
1,46,Beloha,MG,1631725674,-25.1667,45.0500,72.21,57,0,6.76,
2,93,Antakya,TR,1631725688,36.2066,36.1572,78.62,76,0,8.59,
3,259,Kumul,CN,1631725737,42.8000,93.4500,76.33,17,0,8.52,
4,296,Khāsh,IR,1631725748,28.2211,61.2158,78.80,8,0,7.63,
5,350,Yangi Marg`ilon,UZ,1631725767,40.4272,71.7189,75.92,31,0,6.91,


In [20]:
params = {
    "radius": 5000,
    "rankby": "distance",
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_name = requests.get(base_url, params=params)

    hotel_name = hotel_name.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [21]:
hotel_df

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
24,24,Fallon,US,1631725667,39.4735,-118.7774,79.45,20,0,4.47,
46,46,Beloha,MG,1631725674,-25.1667,45.0500,72.21,57,0,6.76,
93,93,Antakya,TR,1631725688,36.2066,36.1572,78.62,76,0,8.59,
259,259,Kumul,CN,1631725737,42.8000,93.4500,76.33,17,0,8.52,
296,296,Khāsh,IR,1631725748,28.2211,61.2158,78.80,8,0,7.63,
350,350,Yangi Marg`ilon,UZ,1631725767,40.4272,71.7189,75.92,31,0,6.91,


In [22]:
# Configure gmaps
gmaps.configure(api_key=g_key)

coordinates = hotel_df[["Latitude", "Longitude"]]


In [23]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [24]:
markers = gmaps.marker_layer(coordinates)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))